# How to use SHERPA simulation code #

At first you need to create a python environment, to be able to run the code.

Then, the steps to run the code are:
- load required libraries
- load basecase emissions and concentrations
- load the source-receptor model to be used
- define reduction areas and percentage of reductions by sector-pollutant
- define output directory
- run the model and inspect results

The example below works for PM2.5 yearly averages, and only for module 1.

The same can be done for yearly averages of PM10, NO2, NO, O3, and for other modules.

All final concentrations are in microg/m3, but O3 that is in ppb.

### Set-up of the python environment

Install a package manager, as i.e. from https://docs.conda.io/en/latest/miniconda.html

In your windows shell, run the following command (to create a python environment, using the packages as in 'environment.yml')  
*conda env create --prefix C:\PATH_ANACONDA\envs\py310-sherpa -f environment.yml*

In your windows shell, activate the environment  
*conda activate py310-sherpa*

### Run module 1
After setting up the environment, you can then run module 1 through the following steps (please note that here below we included some additional python commands, mainly useful to show the content of the different input data required by module 1).

In [ ]:
#import required libraries
import xarray as xr
import pandas as pd
import os
import matplotlib.pyplot as plt
from module1 import module1

In [ ]:
#load basecase emissions 
path_emission_cdf_test = './input/BC_emiss/yea/emep45_cams80_SecEmis.nc'
emis = xr.open_dataset(path_emission_cdf_test)
emis

In [ ]:
#load basecase concentrations
path_base_conc_cdf_test = './input/BC_concs/yea/BCconc_emepV45_cams80_SURF_ug_PM25_rh50.nc'
conc = xr.open_dataset(path_base_conc_cdf_test)
conc

In [ ]:
#load the SRR model
path_model_cdf_test = './input/SRR/yea/SR_SURF_ug_PM25_rh50.nc' 
srr = xr.open_dataset(path_model_cdf_test)
srr

In [ ]:
#define reduction area
path_area_cdf_test = './input/reduction_areas/emiRedOn_emep45_cams80_Germany.nc'#London_emepCams_0_100_FLIP.nc'
area = xr.open_dataset(path_area_cdf_test)
area

In [ ]:
#show selected area
area['AREA'].plot()

In [ ]:
#define percentage reductions
path_reduction_txt_test = './input/reduction_perc/user_reduction_GNFR_all_PPM25.txt'#user_reduction_GNFR_all.txt'
pd.read_csv(path_reduction_txt_test, sep='\t')

In [ ]:
#define output area and downscaling (not implemented now)
path_result_cdf_test = './output/'
downscale_request = 0 # 0 if you do not need downscale, 1 if you need downscale but only for PM concentrations

In [ ]:
#run the model and inspect the results - running time around 1 minute
res = module1(path_emission_cdf_test, path_area_cdf_test, path_reduction_txt_test, path_base_conc_cdf_test, path_model_cdf_test, path_result_cdf_test,
                downscale_request)


In [ ]:
#inspect results, in output file
os.listdir('output')

In [ ]:
#show content of result file
pm25_delta_conc = xr.open_dataset('output/DCconc_emepV434_camsV42_SURF_ug_PM25_rh50.nc')
pm25_delta_conc

In [ ]:
#show delta concentrations
pm25_delta_conc['delta_conc'].plot()
